In [30]:

from __future__ import print_function
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
model = init_chat_model("gemini-2.0-flash",model_provider="google_genai")

In [31]:

def getSuggestions(weatherSummary):
    """
        To produce suggestions for user based on current weather report
    Args:
        weatherSummary : Detailed weather report
    """
    from langchain_core.prompts import ChatPromptTemplate
    prompt_template = ChatPromptTemplate([
        ("system", "You are given weather report, Tell the user how to plan their day accordingly"),
        ("user", "Make suggestions for this report {weatherSummary}")
    ])

    prompt = prompt_template.invoke({
        "weatherSummary" : weatherSummary
    })

    res = model.invoke(prompt)
    return res.content 


In [32]:
def getWeatherInfo(city: str) -> any:
    """
        To get the weather info in the given city

    Args:
        city : CIty in which the weather report to be fetched
    """
    
    import time
    import weatherapi
    from weatherapi.rest import ApiException
    from pprint import pprint
    import datetime

    currentDate = datetime.date.today()
    formatted_date = currentDate.strftime("%Y-%m-%d")
    # Configure API key authorization: ApiKeyAuth
    configuration = weatherapi.Configuration()
    configuration.api_key['key'] = 'e00b8e34d43b4cbea5380937250906'
    # Uncomment below to setup prefix (e.g. Bearer) for API key, if needed
    # configuration.api_key_prefix['key'] = 'Bearer'

    # create an instance of the API class
    api_instance = weatherapi.APIsApi(weatherapi.ApiClient(configuration))
    q = city
    dt = "2025-06-11" 

    try:
        # Astronomy API
        api_response = api_instance.forecast_weather(q, dt)
        return (api_response)
    except ApiException as e:
        print("Exception when calling APIsApi->astronomy: %s\n" % e)

In [33]:
from langchain_core.prompts import ChatPromptTemplate

def makeWeatherReport(weatherReport):
    """
        Based on JSON provided by weather API
        to generate a detailed overview of the climate
    Args:
        weatherReport: JSON file containing all the weather details
    """
    prompt_template = ChatPromptTemplate([
        ("system", "You are a weather man. You will given a detailed JSON weather report. Summarize the important and significant points in detail"),
        ("user", "Give me weather report for {weatherJSON}")
    ]) 
    prompt = prompt_template.invoke({
        "weatherJSON" : weatherReport
    })
    response = model.invoke(prompt)
    return response.content

In [34]:
from langchain_core.runnables import RunnableSequence
from langchain_core.runnables import RunnableLambda
chain = RunnableLambda(getWeatherInfo) | RunnableLambda(makeWeatherReport) | RunnableLambda(getSuggestions)

In [35]:
res = chain.invoke("Madurai")
res

'Okay, here\'s how to plan your day in Madurai on June 11, 2025, based on this weather report:\n\n**Overall:** Be prepared for a day with a high chance of rain and moderately gusty winds. The "feels like" temperature will be higher than the actual temperature during the day, so take precautions to avoid heat exhaustion.\n\n**Morning (Until around Noon):**\n\n*   **Clothing:** Wear light, breathable clothing. Since there\'s a chance of rain, consider clothing that dries quickly.\n*   **Activities:** If you have outdoor plans, be flexible. Carry an umbrella or raincoat.  If possible, schedule outdoor activities for the morning when the "feels like" temperature isn\'t at its peak.\n*   **Sun Protection:** Even with partly cloudy conditions, apply sunscreen, especially if you\'ll be outside during the late morning as the UV index will be moderate. Wear sunglasses and a hat.\n*   **Hydration:** Drink plenty of water throughout the morning to stay hydrated, especially before the "feels like"

In [36]:
with open("../textFiles/weather.txt", "w") as f:
    f.write(res)